In [7]:
#deal 번호를 받아와서
#http://www.tmon.co.kr/api/detail/direct/v1/detailapi/api/detail/board/review/list/4284526834?&page=100&count=100
#reviewContents가 없을 때까지

In [ ]:
import urllib
from bs4 import BeautifulSoup
import requests
import json
import re
import time
import tqdm
from tqdm.notebook import tqdm

def toJson(data, file_path) :
    with open(file_path, 'w', encoding='utf-8') as file :
        json.dump(data, file)
    return 

# txt_file_path_ls = ['productUrlList_간편식_냉장_냉동.txt', 'productUrlList_과일.txt']
txt_file_path_ls = ['test.txt'] #같은 폴더에 넣고 돌릴 것! -> 'review_data_{}.json'.format(txt_file_path[15:-4]으로 저장되기 때문

for txt_file_path in txt_file_path_ls :

    #productUrlList.txt에서 링크 리스트 불러오기
    r = open(txt_file_path, mode='rt', encoding='utf-8')
    productUrlList = r.readlines()

    review_data = []
    error_page_ls = []

    for prod_url in tqdm(productUrlList) :
        prod_idx = re.sub('http://www.tmon.co.kr/deal/','',prod_url)
        prod_idx = re.sub('\n','',prod_idx)
        
        page = 1
        while True: 
            time.sleep(1)
            if page%5 == 0 :
                time.sleep(10)
            url = "http://www.tmon.co.kr/api/detail/direct/v1/detailapi/api/detail/board/review/list/{}?&page={}&count=100".format(prod_idx, page)
            try :
                html = urllib.request.urlopen(url).read()
            except :
                time.sleep(120)
                html = urllib.request.urlopen(url).read()
            soup = BeautifulSoup(html, 'html.parser')
            text = soup.get_text()
#             print(text)
            try :
                json_data = json.loads(text)
            except :
#                 print("error page : {}".format(page))
                error_page_ls.append(url)
                page += 1
                continue
                
            iter_length = len(json_data['data']['reviewContents'])
            
            if iter_length == 0 :
                break
        
            for idx in range(iter_length) :

                #리뷰 고유 번호
                review_idx  = json_data['data']['reviewContents'][idx]['reviewSrl']
                #상품 번호
                prod_idx = prod_idx
                #상품 만족도
                prod_satis = json_data['data']['reviewContents'][idx]['dealGpoint']
                #배송 만족도
                ship_satis = json_data['data']['reviewContents'][idx]['deliveryGpoint']
                #구매한 옵션
                try :
                    select_options = json_data['data']['reviewContents'][idx]['buyOptionList'][0]['optionTitle']
                except :
                    select_options =''
                #구매한 날짜
                try :
                    buy_date = json_data['data']['reviewContents'][idx]['buyOptionList'][0]['buyDate']
                except :
                    buy_date = ''
                #리뷰 작성 날짜
                try :
                    review_date = json_data['data']['reviewContents'][idx]['reviewSurveyList'][0]['updateDt']
                except :
                    review_date = ''
                #사용만족도 옵션1
                try :
                    use_opt1 = json_data['data']['reviewContents'][idx]['reviewSurveyList'][0]['keyword']
                    use_satis1 = json_data['data']['reviewContents'][idx]['reviewSurveyList'][0]['selection']
                    #사용만족도 옵션2
                    use_opt2 = json_data['data']['reviewContents'][idx]['reviewSurveyList'][1]['keyword']
                    use_satis2 = json_data['data']['reviewContents'][idx]['reviewSurveyList'][1]['selection']
                except : 
                    use_opt1 = ''
                    use_satis1 = ''
                    use_opt2 = ''
                    use_satis2 = ''
                #리뷰 내용
                content = json_data['data']['reviewContents'][idx]['contents']
                #user_id
                user_id = json_data['data']['reviewContents'][idx]['whoUpdate']
                #memberSrl
                memberSrl =json_data['data']['reviewContents'][idx]['memberSrl']

                review_data.append({
                        'review_idx' : review_idx,
                        'prod_idx' : prod_idx,
                        'prod_satis' : prod_satis,
                        'ship_satis' : ship_satis,
                        'select_options' : select_options,
                        'buy_date' : buy_date,
                        'review_date' : review_date,
                        'use_opt1' : use_opt1,
                        'use_satis1' : use_satis1,
                        'use_opt2' : use_opt2,
                        'use_satis2' : use_satis2,
                        'content' : content,
                        'user_id' : user_id,
                        'memberSrl' : memberSrl
                    })
                

            page += 1

    toJson(review_data, 'review_data_{}.json'.format(txt_file_path[15:-4]))
    toJson(error_page_ls, 'error_page_{}.json'.format(txt_file_path[15:-4]))

error page : 10


In [28]:
import pandas as pd
data_df = pd.read_json('review_data_.json')
data_df

,review_idx,prod_idx,prod_satis,ship_satis,select_options,buy_date,review_date,use_opt1,use_satis1,use_opt2,use_satis2,content,user_id,memberSrl
0,168445490,2865723582,5,5,04_꿀고구마 정품 대1kg (2개 주문시 5kg 발송),04_꿀고구마 정품 대1kg (2개 주문시 5kg 발송),2020-10-06 12:16:36,맛,맛있어요,신선도,신선해요,잘받았어요잘받았어요,kik*****,18027773
1,168424558,2865723582,3,4,01_★추천★꿀고구마 정품 중1kg(2개 주문시 3kg 발송),01_★추천★꿀고구마 정품 중1kg(2개 주문시 3kg 발송),2020-10-06 10:40:51,맛,보통이에요,신선도,적당해요,사이즈가 살짝 생각과 달랐지만 먹을만했네요,lit******,1348578
2,168421642,2865723582,5,5,01_★추천★꿀고구마 정품 중1kg(2개 주문시 3kg 발송),01_★추천★꿀고구마 정품 중1kg(2개 주문시 3kg 발송),2020-10-06 10:28:29,맛,보통이에요,신선도,적당해요,정말좋습니다 아주좋아요,seo*****,64600294
3,168421630,2865723582,5,5,01_★추천★꿀고구마 정품 중1kg(2개 주문시 3kg 발송),01_★추천★꿀고구마 정품 중1kg(2개 주문시 3kg 발송),2020-10-06 10:28:29,맛,보통이에요,신선도,적당해요,정말좋습니다 아주좋아요,seo*****,64600294
4,168421198,2865723582,5,5,01_★추천★꿀고구마 정품 중1kg(2개 주문시 3kg 발송),01_★추천★꿀고구마 정품 중1kg(2개 주문시 3kg 발송),2020-10-06 10:26:54,맛,보통이에요,신선도,적당해요,정말좋습니다 아주좋아요,seo*****,64600294
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41577,28808826,617272158,5,5,A04. 목심 보쌈용 500g,A04. 목심 보쌈용 500g,,,,,,"예전부터자주 사먹던거라.. 만족합니다.포장도 깔끔, 배송도 시원~돈까스도 만들고, ...",cry***,18480649
41578,28581426,617272158,5,5,A01. 삼겹살 구이용 500g,A01. 삼겹살 구이용 500g,,,,,,신선하고 맛있어요,qor*********,16133161
41579,28325694,617272158,5,5,A03. 목심 구이용 500g,A03. 목심 구이용 500g,,,,,,여기최고야 단골됬음 너무맛있어서 하루한끼는여기돼지고기로먹음 배송은처음시켰을때5일걸렸...,rla*******,72381618
41580,28324662,617272158,4,4,A02. 삼겹살 보쌈용 500g,A02. 삼겹살 보쌈용 500g,,,,,,좋아여~ 맛있게 잘먹었습니다~,bst****,24400701
